In [99]:
import os
import random
import itertools
import re

# 基本的なライブラリ
import numpy as np
import pandas as pd
from scipy import stats

# 描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import pydotplus
from IPython.display import Image, HTML
from six import StringIO
from ipywidgets import interact, FloatSlider
from dtreeviz.trees import *

# データセット分割
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    ShuffleSplit,
    StratifiedShuffleSplit,
)

# 補完
from sklearn.experimental import (
    enable_iterative_imputer,
)  # IterativeImputerをimportするために必要
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# エンコード
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

# サンプリング
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 特徴量選択
from sklearn.feature_selection import (
    VarianceThreshold,
    GenericUnivariateSelect,
    f_classif,
    mutual_info_classif,
    chi2,
)
from boruta import BorutaPy
from libraries.mrmr import mrmr

# 学習中
import optuna
from tqdm import tqdm
from sklearn.model_selection import learning_curve, cross_validate, cross_val_score

# 評価指標
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# models
from lightgbm import LGBMClassifier
import xgboost as xgb

# XAI
import shap

import warnings


# config python file
import config

SEED = config.SEED
INDEX_MICROARRAY = config.INDEX_MICROARRAY
from functions import *

fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

# データ読み込み

In [100]:
X_dict = pickle_load(config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/X_dict.pickle")
y_dict = pickle_load(config.INTERIM_PREPROCESSED_PROGNOSIS_CROSS_DIR + "/y_dict.pickle")

print(X_dict.keys())

dict_keys(['mrna_agilent_microarray_zscores_ref_all_samples', 'mrna_agilent_microarray', 'mrna_agilent_microarray_zscores_ref_diploid_samples'])


In [101]:
X_train = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["X15"]
y_train = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["train"]["y15"]
X_val = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["X15"]
y_val = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["mrmr"]["val"]["y15"]

assert X_train.shape[0] == y_train.shape[0], "train size is incorrect"
assert X_val.shape[0] == y_val.shape[0], "test size is incorrect"
assert X_train.shape[1] == X_val.shape[1], "feature size is incorrect"

# accuracyの表示
print("----------" * 10)
print("使用特徴量：", X_train.columns)
print("学習サンプルサイズ：", X_train.shape)
display("ラベル比率：", y_train.value_counts())

----------------------------------------------------------------------------------------------------
使用特徴量： Index(['RACGAP1', 'CYP2D6', 'PTX4', 'BIK', 'ENC1', 'DENND6A', 'STAT5B',
       'RNU11', 'DNAJB7', 'PPIL3', 'UHRF1', 'RBBP8', 'MIR137HG', 'GABRB1',
       'TOR1B', 'MST1', 'NMRK1', 'FGF13', 'HIST1H2BF', 'SPATA4', 'TBC1D31',
       'USP30', 'S100P', 'APLN', 'MTG2', 'GAL3ST4', 'FGD3', 'PJVK', 'INTS10',
       'CBX7', 'FCN2', 'RNASE9', 'SPATA18', 'ANKAR', 'SULT4A1', 'TSPYL6',
       'IGDCC4', 'TMEM26', 'PDK3', 'AURKA', 'PRR21', 'DNAAF1', 'GPR151',
       'GASK1A', 'NLRX1', 'RAB3B', 'HS3ST1', 'PKMYT1', 'CATSPERB'],
      dtype='object')
学習サンプルサイズ： (865, 49)


'ラベル比率：'

1    461
0    404
Name: OS_15years, dtype: int64

# 最適化(all data)

分類器を学習させ、パラメータのチューニングを行い、高い予測精度を目指す。

In [103]:
optim_values_path = (
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR + "/microarray_optim_values.pickle"
)
optim_params_path = (
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR + "/microarray_optim_params.pickle"
)
dict_optim_values = (
    pickle_load(optim_values_path) if os.path.exists(optim_values_path) else {}
)

dict_optim_params = (
    pickle_load(optim_params_path) if os.path.exists(optim_params_path) else {}
)
dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]] = dict()
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]] = dict()

## Optuna

### Random Forest

In [107]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 50, log=True),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 100),
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = RandomForestClassifier(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"] = dict()
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"] = dict()

study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study.optimize(objective, n_trials=100, show_progress_bar=True)
dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"][
    year
] = study.best_trial.values
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"][
    year
] = study.best_trial.params

[I 2022-09-14 10:44:02,119] A new study created in memory with name: no-name-63563ad6-a7f6-4a4d-8997-b1d6ba87e3c0


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-14 10:44:04,462] Trial 0 finished with value: 0.73761839754221 and parameters: {'n_estimators': 548, 'max_depth': 4, 'max_leaf_nodes': 44}. Best is trial 0 with value: 0.73761839754221.
[I 2022-09-14 10:44:06,609] Trial 1 finished with value: 0.7158790545918869 and parameters: {'n_estimators': 847, 'max_depth': 2, 'max_leaf_nodes': 14}. Best is trial 0 with value: 0.73761839754221.
[I 2022-09-14 10:44:08,337] Trial 2 finished with value: 0.7442350338072542 and parameters: {'n_estimators': 674, 'max_depth': 27, 'max_leaf_nodes': 15}. Best is trial 2 with value: 0.7442350338072542.
[I 2022-09-14 10:44:09,866] Trial 3 finished with value: 0.7491242266568306 and parameters: {'n_estimators': 579, 'max_depth': 34, 'max_leaf_nodes': 22}. Best is trial 3 with value: 0.7491242266568306.
[I 2022-09-14 10:44:10,215] Trial 4 finished with value: 0.7087875516068863 and parameters: {'n_estimators': 193, 'max_depth': 2, 'max_leaf_nodes': 23}. Best is trial 3 with value: 0.7491242266568306.

### LogisticRegression

In [108]:
def objective(trial):
    # ランダムフォレストのパラメータチューニング
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "C": trial.suggest_float("C", 1e-5, 1e5),
        "max_iter": 1000,
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = LogisticRegression(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=10, scoring=make_scorer(f1_score)
    )
    return score.mean()


dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"] = dict()
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"] = dict()

study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=SEED)
)
study.optimize(objective, n_trials=10)
dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"][
    year
] = study.best_trial.values
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"][
    year
] = study.best_trial.params

[I 2022-09-14 10:46:46,846] A new study created in memory with name: no-name-d99bf388-b306-482f-adc2-cd649dcbab3a
[W 2022-09-14 10:46:46,864] Trial 0 failed, because the objective function returned nan.
[W 2022-09-14 10:46:46,880] Trial 1 failed, because the objective function returned nan.
[I 2022-09-14 10:46:46,910] Trial 2 finished with value: 0.759381025747212 and parameters: {'penalty': 'l2', 'C': 13670.65897712823}. Best is trial 2 with value: 0.759381025747212.
[I 2022-09-14 10:46:46,942] Trial 3 finished with value: 0.759381025747212 and parameters: {'penalty': 'l2', 'C': 20920.212219626937}. Best is trial 2 with value: 0.759381025747212.
[W 2022-09-14 10:46:46,957] Trial 4 failed, because the objective function returned nan.
[W 2022-09-14 10:46:46,976] Trial 5 failed, because the objective function returned nan.
[W 2022-09-14 10:46:46,997] Trial 6 failed, because the objective function returned nan.
[W 2022-09-14 10:46:47,014] Trial 7 failed, because the objective function ret

### lightGBM

In [109]:
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "class_weight": "balanced",
        "random_state": SEED,
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=10)
    return score.mean()


dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True)
dict_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
    year
] = study.best_trial.values
dict_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
    year
] = study.best_trial.params

[I 2022-09-14 10:46:47,086] A new study created in memory with name: no-name-77912108-b344-4e23-8ecf-c6a4e8615b14


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-14 10:46:47,579] Trial 0 finished with value: 0.7341753541833734 and parameters: {'lambda_l1': 3.3136042776924254e-07, 'lambda_l2': 1.2401984848934321e-07, 'max_bin': 171, 'num_leaves': 44, 'learning_rate': 0.06619941969371747, 'n_estimators': 82}. Best is trial 0 with value: 0.7341753541833734.
[I 2022-09-14 10:46:48,920] Trial 1 finished with value: 0.6878909382518044 and parameters: {'lambda_l1': 0.01636720148539928, 'lambda_l2': 8.166364726682683e-07, 'max_bin': 219, 'num_leaves': 38, 'learning_rate': 0.0018280091709739084, 'n_estimators': 268}. Best is trial 0 with value: 0.7341753541833734.
[I 2022-09-14 10:46:49,539] Trial 2 finished with value: 0.731769580326116 and parameters: {'lambda_l1': 3.0383979553093368, 'lambda_l2': 0.007972309662091625, 'max_bin': 205, 'num_leaves': 20, 'learning_rate': 0.04997995140594941, 'n_estimators': 612}. Best is trial 0 with value: 0.7341753541833734.
[I 2022-09-14 10:46:50,100] Trial 3 finished with value: 0.7178829190056135 and par

## save params dict as pickle

In [111]:
make_dir(config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR)
pickle_dump(
    dict_optim_values,
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR + "/microarray_optim_values.pickle",
)
pickle_dump(
    dict_optim_params,
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR + "/microarray_optim_params.pickle",
)

# 最適化(subtypes毎)

分類器を学習させ、パラメータのチューニングを行い、高い予測精度を目指す。

In [88]:
subtype_optim_values_path = (
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR
    + "/microarray_subtype_optim_values.pickle"
)
subtype_optim_param_path = (
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR
    + "/microarray_subtype_optim_params.pickle"
)

dict_subtype_optim_values = (
    pickle_load(subtype_optim_values_path)
    if os.path.exists(subtype_optim_values_path)
    else {}
)
dict_subtype_optim_params = (
    pickle_load(subtype_optim_param_path)
    if os.path.exists(subtype_optim_param_path)
    else {}
)

dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]] = dict()
dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]] = dict()

## RandomForest

In [90]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 50, log=True),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 100),
        "class_weight": trial.suggest_categorical(
            "class_weight", ["balanced", "balanced_subsample"]
        ),
        "random_state": SEED,
    }

    clf = RandomForestClassifier(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()


optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]


dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"] = dict()
dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"] = dict()
for year in range(15, 16, 5):  # 予後年数毎のループ
    for subtype in subtypes:
        dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"][
            subtype
        ] = dict()
        dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"][
            subtype
        ] = dict()
        prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
        prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
        X_train_subtype = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
            "claudin_subtype"
        ]["mrmr"]["train"][prognosis_Xlabel]
        y_train_subtype = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
            "claudin_subtype"
        ]["mrmr"]["train"][prognosis_ylabel]
        assert (
            X_train_subtype.shape[0] == y_train_subtype.shape[0]
        ), "train size is incorrect"

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=100, show_progress_bar=True)
        dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"][
            subtype
        ][year] = study.best_trial.values
        dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["rf"][
            subtype
        ][year] = study.best_trial.params

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-14 09:37:55,610] Trial 0 finished with value: 0.8111111111111111 and parameters: {'n_estimators': 777, 'max_depth': 44, 'max_leaf_nodes': 21, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.8111111111111111.
[I 2022-09-14 09:37:56,656] Trial 1 finished with value: 0.788888888888889 and parameters: {'n_estimators': 710, 'max_depth': 3, 'max_leaf_nodes': 77, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.8111111111111111.
[I 2022-09-14 09:37:56,763] Trial 2 finished with value: 0.7555555555555555 and parameters: {'n_estimators': 56, 'max_depth': 17, 'max_leaf_nodes': 62, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.8111111111111111.
[I 2022-09-14 09:37:57,979] Trial 3 finished with value: 0.8111111111111111 and parameters: {'n_estimators': 843, 'max_depth': 12, 'max_leaf_nodes': 89, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.8111111111111111.
[I 2022-09-14 09:37:58,601] Trial 4 finished with val

[I 2022-09-14 09:39:45,059] A new study created in memory with name: no-name-35412aee-7d32-4881-9ed0-decf2176284a


[I 2022-09-14 09:39:45,056] Trial 99 finished with value: 0.8111111111111111 and parameters: {'n_estimators': 849, 'max_depth': 24, 'max_leaf_nodes': 83, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.8111111111111111.


  0%|          | 0/100 [00:00<?, ?it/s]

...

[I 2022-09-14 09:43:52,403] Trial 0 finished with value: 0.86 and parameters: {'n_estimators': 262, 'max_depth': 20, 'max_leaf_nodes': 18, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.86.
[I 2022-09-14 09:43:52,677] Trial 1 finished with value: 0.8966666666666668 and parameters: {'n_estimators': 149, 'max_depth': 13, 'max_leaf_nodes': 79, 'class_weight': 'balanced_subsample'}. Best is trial 1 with value: 0.8966666666666668.
[I 2022-09-14 09:43:53,725] Trial 2 finished with value: 0.8766666666666667 and parameters: {'n_estimators': 639, 'max_depth': 3, 'max_leaf_nodes': 29, 'class_weight': 'balanced_subsample'}. Best is trial 1 with value: 0.8966666666666668.
[I 2022-09-14 09:43:53,885] Trial 3 finished with value: 0.8266666666666665 and parameters: {'n_estimators': 86, 'max_depth': 6, 'max_leaf_nodes': 7, 'class_weight': 'balanced_subsample'}. Best is trial 1 with value: 0.8966666666666668.
[I 2022-09-14 09:43:54,751] Trial 4 finished with value: 0.8800000000000

[I 2022-09-14 09:45:15,098] A new study created in memory with name: no-name-b3547228-5eac-4fad-9c93-e685c5cfa805


[I 2022-09-14 09:45:15,095] Trial 99 finished with value: 0.8966666666666668 and parameters: {'n_estimators': 811, 'max_depth': 12, 'max_leaf_nodes': 36, 'class_weight': 'balanced_subsample'}. Best is trial 1 with value: 0.8966666666666668.


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-14 09:45:15,250] Trial 0 finished with value: 0.7718181818181817 and parameters: {'n_estimators': 76, 'max_depth': 15, 'max_leaf_nodes': 97, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.7718181818181817.
[I 2022-09-14 09:45:16,821] Trial 1 finished with value: 0.7718181818181817 and parameters: {'n_estimators': 921, 'max_depth': 2, 'max_leaf_nodes': 60, 'class_weight': 'balanced_subsample'}. Best is trial 0 with value: 0.7718181818181817.
[I 2022-09-14 09:45:17,327] Trial 2 finished with value: 0.7518181818181817 and parameters: {'n_estimators': 363, 'max_depth': 36, 'max_leaf_nodes': 9, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7718181818181817.
[I 2022-09-14 09:45:18,555] Trial 3 finished with value: 0.7618181818181818 and parameters: {'n_estimators': 866, 'max_depth': 4, 'max_leaf_nodes': 69, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7718181818181817.
[I 2022-09-14 09:45:19,434] Trial 4 finished with value: 0.75181

## LogitsticRegression

In [91]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "C": trial.suggest_float("C", 1e-5, 1e5),
        "max_iter": 1000,
        "class_weight": "balanced",
        "random_state": SEED,
    }

    clf = LogisticRegression(**params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()

In [92]:
optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]
dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"] = dict()
dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"] = dict()
for year in range(15, 16, 5):  # 予後年数毎のループ
    print("====={0:0=2}".format(year) * 10)
    for subtype in subtypes:
        print("----------" * 10)
        print(subtype)
        dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"][
            subtype
        ] = dict()
        dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"][
            subtype
        ] = dict()
        prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
        prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
        X_train_subtype = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
            "claudin_subtype"
        ]["mrmr"]["train"][prognosis_Xlabel]
        y_train_subtype = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
            "claudin_subtype"
        ]["mrmr"]["train"][prognosis_ylabel]
        assert (
            X_train_subtype.shape[0] == y_train_subtype.shape[0]
        ), "train size is incorrect"

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=100, show_progress_bar=True)
        dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"][
            subtype
        ][year] = study.best_trial.values
        dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lr"][
            subtype
        ][year] = study.best_trial.params

=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15
----------------------------------------------------------------------------------------------------
claudin-low


  0%|          | 0/100 [00:00<?, ?it/s]

[W 2022-09-14 09:45:55,846] Trial 0 failed, because the objective function returned nan.
[I 2022-09-14 09:45:55,864] Trial 1 finished with value: 0.8777777777777779 and parameters: {'penalty': 'l2', 'C': 29574.76829504445}. Best is trial 1 with value: 0.8777777777777779.
[W 2022-09-14 09:45:55,876] Trial 2 failed, because the objective function returned nan.
[W 2022-09-14 09:45:55,889] Trial 3 failed, because the objective function returned nan.
[I 2022-09-14 09:45:55,908] Trial 4 finished with value: 0.8777777777777779 and parameters: {'penalty': 'l2', 'C': 1381.8206006217147}. Best is trial 1 with value: 0.8777777777777779.
[W 2022-09-14 09:45:55,920] Trial 5 failed, because the objective function returned nan.
[W 2022-09-14 09:45:55,934] Trial 6 failed, because the objective function returned nan.
[W 2022-09-14 09:45:55,945] Trial 7 failed, because the objective function returned nan.
[W 2022-09-14 09:45:55,957] Trial 8 failed, because the objective function returned nan.
[W 2022-09

[I 2022-09-14 09:45:57,337] A new study created in memory with name: no-name-e43fe8d1-7a1c-4ed6-a005-2642931848b3


[W 2022-09-14 09:45:57,335] Trial 99 failed, because the objective function returned nan.
----------------------------------------------------------------------------------------------------
LumA


...

[W 2022-09-14 09:46:02,085] Trial 0 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,096] Trial 1 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,109] Trial 2 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,121] Trial 3 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,134] Trial 4 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,146] Trial 5 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,157] Trial 6 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,171] Trial 7 failed, because the objective function returned nan.
[W 2022-09-14 09:46:02,182] Trial 8 failed, because the objective function returned nan.
[I 2022-09-14 09:46:02,200] Trial 9 finished with value: 1.0 and parameters: {'penalty': 'l2', 'C': 86391.59584084364}. Best is trial 9 with value: 1.0.
[W 2022-09-14 09:46:02,212] Trial 10 failed, b

[I 2022-09-14 09:46:03,542] A new study created in memory with name: no-name-1d4cb39f-38ab-44f9-8658-686b81f63619


[W 2022-09-14 09:46:03,541] Trial 99 failed, because the objective function returned nan.
----------------------------------------------------------------------------------------------------
Basal


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-09-14 09:46:03,577] Trial 0 finished with value: 0.8454545454545455 and parameters: {'penalty': 'l2', 'C': 61226.78211720135}. Best is trial 0 with value: 0.8454545454545455.
[W 2022-09-14 09:46:03,588] Trial 1 failed, because the objective function returned nan.
[I 2022-09-14 09:46:03,606] Trial 2 finished with value: 0.8454545454545455 and parameters: {'penalty': 'l2', 'C': 64128.073324100355}. Best is trial 0 with value: 0.8454545454545455.
[W 2022-09-14 09:46:03,617] Trial 3 failed, because the objective function returned nan.
[W 2022-09-14 09:46:03,630] Trial 4 failed, because the objective function returned nan.
[I 2022-09-14 09:46:03,648] Trial 5 finished with value: 0.8454545454545455 and parameters: {'penalty': 'l2', 'C': 32420.829144535233}. Best is trial 0 with value: 0.8454545454545455.
[W 2022-09-14 09:46:03,661] Trial 6 failed, because the objective function returned nan.
[W 2022-09-14 09:46:03,671] Trial 7 failed, because the objective function returned nan.
[W 2

## lightGBM

In [93]:
def objective(trial):
    warnings.simplefilter("ignore")
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "max_bin": trial.suggest_int("max_bin", 100, 300),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "class_weight": "balanced",
        "random_state": SEED,
    }
    clf = LGBMClassifier(boosting_type="gbdt", **params)
    # 10分割交差検証によるテストデータのaccuracyの出力
    score = cross_val_score(clf, X_train_subtype, y_train_subtype, n_jobs=-1, cv=10)
    return score.mean()

In [94]:
optuna.logging.disable_default_handler()
warnings.simplefilter("ignore")
subtypes = [
    "claudin-low",
    "LumA",
    "LumB",
    "Her2",
    "Normal",
    "Basal",
]
dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()
dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"] = dict()
for year in range(15, 16, 5):  # 予後年数毎のループ
    print("====={0:0=2}".format(year) * 10)
    for subtype in tqdm(subtypes):
        print("----------" * 10)
        print(subtype)
        dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
            subtype
        ] = dict()
        dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
            subtype
        ] = dict()
        prognosis_Xlabel = "X{0:0=2}_{1}".format(year, subtype)
        prognosis_ylabel = "y{0:0=2}_{1}".format(year, subtype)
        X_train_subtype = X_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
            "claudin_subtype"
        ]["mrmr"]["train"][prognosis_Xlabel]
        y_train_subtype = y_dict[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]][
            "claudin_subtype"
        ]["mrmr"]["train"][prognosis_ylabel]
        assert (
            X_train_subtype.shape[0] == y_train_subtype.shape[0]
        ), "train size is incorrect"

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=100, show_progress_bar=True)
        dict_subtype_optim_values[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
            subtype
        ][year] = study.best_trial.values
        dict_subtype_optim_params[config.SET_NAME_MICROARRAY[INDEX_MICROARRAY]]["lgb"][
            subtype
        ][year] = study.best_trial.params

=====15=====15=====15=====15=====15=====15=====15=====15=====15=====15


  0%|                                                               | 0/6 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
claudin-low


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|                                                               | 0/6 [00:00<?, ?it/s]

...

[I 2022-09-14 09:47:37,754] Trial 94 finished with value: 0.760909090909091 and parameters: {'lambda_l1': 1.9494296525595997e-07, 'lambda_l2': 3.349006251080385e-06, 'max_bin': 163, 'num_leaves': 36, 'learning_rate': 0.0051102299919749865, 'n_estimators': 812}. Best is trial 0 with value: 0.7790909090909091.
[I 2022-09-14 09:47:37,898] Trial 95 finished with value: 0.77 and parameters: {'lambda_l1': 0.0296839304869693, 'lambda_l2': 3.429304337851505e-05, 'max_bin': 168, 'num_leaves': 38, 'learning_rate': 0.0036029602239411474, 'n_estimators': 947}. Best is trial 0 with value: 0.7790909090909091.


 83%|█████████████████████████████████████████████▊         | 5/6 [01:33<00:15, 15.18s/it]

[I 2022-09-14 09:47:38,034] Trial 96 finished with value: 0.7418181818181818 and parameters: {'lambda_l1': 1.638957969789321e-06, 'lambda_l2': 0.0015293157872405263, 'max_bin': 172, 'num_leaves': 36, 'learning_rate': 0.00310619978730405, 'n_estimators': 876}. Best is trial 0 with value: 0.7790909090909091.
[I 2022-09-14 09:47:38,184] Trial 97 finished with value: 0.7318181818181817 and parameters: {'lambda_l1': 0.8802604037124849, 'lambda_l2': 2.573477773923904e-05, 'max_bin': 171, 'num_leaves': 41, 'learning_rate': 0.002409514454273085, 'n_estimators': 992}. Best is trial 0 with value: 0.7790909090909091.
[I 2022-09-14 09:47:38,231] Trial 98 finished with value: 0.61 and parameters: {'lambda_l1': 9.569870903457095e-08, 'lambda_l2': 0.06219711206324069, 'max_bin': 136, 'num_leaves': 28, 'learning_rate': 0.006235900567480612, 'n_estimators': 93}. Best is trial 0 with value: 0.7790909090909091.


100%|███████████████████████████████████████████████████████| 6/6 [01:33<00:00, 15.55s/it]

[I 2022-09-14 09:47:38,370] Trial 99 finished with value: 0.77 and parameters: {'lambda_l1': 1.7094695287157283e-07, 'lambda_l2': 0.0005841681812943088, 'max_bin': 176, 'num_leaves': 40, 'learning_rate': 0.004055499662868837, 'n_estimators': 962}. Best is trial 0 with value: 0.7790909090909091.


## save param info

In [97]:
make_dir(config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR)
pickle_dump(
    dict_subtype_optim_values,
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR
    + "/microarray_subtype_optim_values.pickle",
)
pickle_dump(
    dict_subtype_optim_params,
    config.INTERIM_TUNING_PROGNOSIS_CROSS_DIR
    + "/microarray_subtype_optim_params.pickle",
)

[LightGBM] [Warning] lambda_l1 is set=5.713437181623996, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.713437181623996
[LightGBM] [Warning] lambda_l2 is set=1.3738757580639929, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3738757580639929
[LightGBM] [Warning] lambda_l1 is set=4.420032000266733e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.420032000266733e-05
[LightGBM] [Warning] lambda_l2 is set=0.5098090675057648, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5098090675057648
[LightGBM] [Warning] lambda_l1 is set=2.4310066485405403e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.4310066485405403e-05
[LightGBM] [Warning] lambda_l2 is set=3.339302961107622, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.339302961107622
[LightGBM] [Warning] lambda_l1 is set=1.466167889606233e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.466167889606233e-06
[LightGBM] [Warning] lambda_l2 is set=0.5748314129